In [1]:
class CFG:
    apex=True
    print_freq=100
    num_workers=4
    model="microsoft/deberta-v3-small"
    scheduler='cosine' # ['linear', 'cosine']
    num_warmup_steps=0
    epochs=5
    lr = 2e-5
    eps=1e-6
    batch_size=12
    fc_dropout=0.2
    max_len=512
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1000
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]

In [2]:
import os
import gc
import ast
import itertools
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import GroupKFold

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
os.system('python -m pip uninstall transformers -y')
os.system('python -m pip install -U transformers==4.28.1 tokenizers==0.13.3 accelerate==0.20.3')
# os.system('python -m pip install --no-index --find-links=../input/nbme-pip-wheels transformers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_cosine_schedule_with_warmup

Found existing installation: transformers 4.12.5
Uninstalling transformers-4.12.5:
  Successfully uninstalled transformers-4.12.5


  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.1.2
    Uninstalling huggingface-hub-0.1.2:
      Successfully uninstalled huggingface-hub-0.1.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cached-path 0.3.2 requires huggingface-hub<0.2.0,>=0.0.12, but you have huggingface-hub 0.16.4 which is incompatible.
allennlp 2.8.0 requires transformers<4.13,>=4.1, but you have transformers 4.28.1 which is incompatible.


tokenizers.__version__: 0.13.3
transformers.__version__: 4.28.1


In [3]:
import subprocess

subprocess.run([
    "pip", "download",
    "transformers==4.28.1",
    "tokenizers==0.13.3",
    "accelerate==0.20.3",
    "-d", "./downloaded_packages"
])

  Using cached transformers-4.28.1-py3-none-any.whl (7.0 MB)
  Using cached tokenizers-0.13.3-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Using cached accelerate-0.20.3-py3-none-any.whl (227 kB)
  Using cached huggingface_hub-0.16.4-py3-none-any.whl (268 kB)
Saved ./downloaded_packages/transformers-4.28.1-py3-none-any.whl
Saved ./downloaded_packages/tokenizers-0.13.3-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Saved ./downloaded_packages/accelerate-0.20.3-py3-none-any.whl
Saved ./downloaded_packages/huggingface_hub-0.16.4-py3-none-any.whl
Saved ./downloaded_packages/numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
Saved ./downloaded_packages/packaging-24.0-py3-none-any.whl
Saved ./downloaded_packages/PyYAML-6.0.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Saved ./downloaded_packages/regex-2024.4.16-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Saved ./downloaded_packages/torch-1.13.1-cp37-cp37m-many

CompletedProcess(args=['pip', 'download', 'transformers==4.28.1', 'tokenizers==0.13.3', 'accelerate==0.20.3', '-d', './downloaded_packages'], returncode=0)

In [4]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed=CFG.seed)

In [5]:
def micro_f1(preds, truths):
    """
    Micro f1 on binary arrays.

    Args:
        preds (list of lists of ints): Predictions.
        truths (list of lists of ints): Ground truths.

    Returns:
        float: f1 score.
    """
    # Micro : aggregating over all instances
    preds = np.concatenate(preds)
    truths = np.concatenate(truths)
    return f1_score(truths, preds)


def spans_to_binary(spans, length=None):
    """
    Converts spans to a binary array indicating whether each character is in the span.

    Args:
        spans (list of lists of two ints): Spans.

    Returns:
        np array [length]: Binarized spans.
    """
    length = np.max(spans) if length is None else length
    binary = np.zeros(length)
    for start, end in spans:
        binary[start:end] = 1
    return binary


def span_micro_f1(preds, truths):
    """
    Micro f1 on spans.

    Args:
        preds (list of lists of two ints): Prediction spans.
        truths (list of lists of two ints): Ground truth spans.

    Returns:
        float: f1 score.
    """
    bin_preds = []
    bin_truths = []
    for pred, truth in zip(preds, truths):
        if not len(pred) and not len(truth):
            continue
        length = max(np.max(pred) if len(pred) else 0, np.max(truth) if len(truth) else 0)
        bin_preds.append(spans_to_binary(pred, length))
        bin_truths.append(spans_to_binary(truth, length))
    return micro_f1(bin_preds, bin_truths)

In [6]:
def create_labels_for_scoring(df):
    # example: ['0 1', '3 4'] -> ['0 1; 3 4']
    df['location_for_create_labels'] = [ast.literal_eval(f'[]')] * len(df)
    for i in range(len(df)):
        lst = df.loc[i, 'location']
        if lst:
            new_lst = ';'.join(lst)
            df.loc[i, 'location_for_create_labels'] = ast.literal_eval(f'[["{new_lst}"]]')
    # create labels
    truths = []
    for location_list in df['location_for_create_labels'].values:
        truth = []
        if len(location_list) > 0:
            location = location_list[0]
            for loc in [s.split() for s in location.split(';')]:
                start, end = int(loc[0]), int(loc[1])
                truth.append([start, end])
        truths.append(truth)
    return truths


def get_char_probs(texts, predictions, tokenizer):
    results = [np.zeros(len(t)) for t in texts]
    for i, (text, prediction) in enumerate(zip(texts, predictions)):
        encoded = tokenizer(text, 
                            add_special_tokens=True,
                            return_offsets_mapping=True)
        for idx, (offset_mapping, pred) in enumerate(zip(encoded['offset_mapping'], prediction)):
            start = offset_mapping[0]
            end = offset_mapping[1]
            results[i][start:end] = pred
    return results


def get_results(char_probs, th=0.5):
    results = []
    for char_prob in char_probs:
        result = np.where(char_prob >= th)[0] + 1
        result = [list(g) for _, g in itertools.groupby(result, key=lambda n, c=itertools.count(): n - next(c))]
        result = [f"{min(r)} {max(r)}" for r in result]
        result = ";".join(result)
        results.append(result)
    return results


def get_predictions(results):
    predictions = []
    for result in results:
        prediction = []
        if result != "":
            for loc in [s.split() for s in result.split(';')]:
                start, end = int(loc[0]), int(loc[1])
                prediction.append([start, end])
        predictions.append(prediction)
    return predictions

In [7]:
def get_score(y_true, y_pred):
    score = span_micro_f1(y_true, y_pred)
    return score

In [8]:
data_dir = "../input/nbme-score-clinical-patient-notes/"
train = pd.read_csv(f'{data_dir}train.csv')
features = pd.read_csv(f'{data_dir}features.csv')
patient_notes = pd.read_csv(f'{data_dir}patient_notes.csv')
def preprocess_features(features):
    features.loc[27, 'feature_text'] = "last-pap-smear-1-year-ago"
    return features
features = preprocess_features(features)

train['annotation'] = train['annotation'].apply(ast.literal_eval)
train['location'] = train['location'].apply(ast.literal_eval)

print(f"train.shape: {train.shape}")
display(train.head())
print(f"features.shape: {features.shape}")
display(features.head())
print(f"patient_notes.shape: {patient_notes.shape}")
display(patient_notes.head())

train.shape: (14300, 6)


,id,case_num,pn_num,feature_num,annotation,location
0,00016_000,0,16,0,[dad with recent heart attcak],[696 724]
1,00016_001,0,16,1,"[mom with ""thyroid disease]",[668 693]
2,00016_002,0,16,2,[chest pressure],[203 217]
3,00016_003,0,16,3,"[intermittent episodes, episode]","[70 91, 176 183]"
4,00016_004,0,16,4,[felt as if he were going to pass out],[222 258]


features.shape: (143, 3)


,feature_num,case_num,feature_text
0,0,0,Family-history-of-MI-OR-Family-history-of-myoc...
1,1,0,Family-history-of-thyroid-disorder
2,2,0,Chest-pressure
3,3,0,Intermittent-symptoms
4,4,0,Lightheaded


patient_notes.shape: (42146, 3)


,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student heal..."
1,1,0,17 yo male with recurrent palpitations for the...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...
4,4,0,17yo male with no pmh here for evaluation of p...


In [9]:
# incorrect annotation
corrections = {
    338: {"annotation": [["father heart attack"]], "location": [["764 783"]]},
    621: {"annotation": [["for the last 2-3 months"]], "location": [["77 100"]]},
    655: {"annotation": [["no heat intolerance"], ["no cold intolerance"]], "location": [["285 292;301 312"], ["285 287;296 312"]]},
    1262: {"annotation": [["mother thyroid problem"]], "location": [["551 557;565 580"]]},
    1265: {"annotation": [['felt like he was going to "pass out"']], "location": [["131 135;181 212"]]},
    1396: {"annotation": [["stool , with no blood"]], "location": [["259 280"]]},
    1591: {"annotation": [["diarrhoe non blooody"]], "location": [["176 184;201 212"]]},
    1615: {"annotation": [["diarrhea for last 2-3 days"]], "location": [["249 257;271 288"]]},
    1664: {"annotation": [["no vaginal discharge"]], "location": [["822 824;907 924"]]},
    1714: {"annotation": [["started about 8-10 hours ago"]], "location": [["101 129"]]},
    1929: {"annotation": [["no blood in the stool"]], "location": [["531 539;549 561"]]},
    2134: {"annotation": [["last sexually active 9 months ago"]], "location": [["540 560;581 593"]]},
    2191: {"annotation": [["right lower quadrant pain"]], "location": [["32 57"]]},
    2553: {"annotation": [["diarrhoea no blood"]], "location": [["308 317;376 384"]]},
    3124: {"annotation": [["sweating"]], "location": [["549 557"]]},
    3858: {"annotation": [["previously as regular"], ["previously eveyr 28-29 days"], ["previously lasting 5 days"], ["previously regular flow"]],
           "location": [["102 123"], ["102 112;125 141"], ["102 112;143 157"], ["102 112;159 171"]]},
    4373: {"annotation": [["for 2 months"]], "location": [["33 45"]]},
    4763: {"annotation": [["35 year old"]], "location": [["5 16"]]},
    4782: {"annotation": [["darker brown stools"]], "location": [["175 194"]]},
    4908: {"annotation": [["uncle with peptic ulcer"]], "location": [["700 723"]]},
    6016: {"annotation": [["difficulty falling asleep"]], "location": [["225 250"]]},
    6192: {"annotation": [["helps to take care of aging mother and in-laws"]], "location": [["197 218;236 260"]]},
    6380: {"annotation": [["No hair changes"], ["No skin changes"], ["No GI changes"], ["No palpitations"], ["No excessive sweating"]],
           "location": [["480 482;507 519"], ["480 482;499 503;512 519"], ["480 482;521 531"], ["480 482;533 545"], ["480 482;564 582"]]},
    6562: {"annotation": [["stressed due to taking care of her mother"], ["stressed due to taking care of husbands parents"]],
           "location": [["290 320;327 337"], ["290 320;342 358"]]},
    6862: {"annotation": [["stressor taking care of many sick family members"]], "location": [["288 296;324 363"]]},
    7022: {"annotation": [["heart started racing and felt numbness for the 1st time in her finger tips"]], "location": [["108 182"]]},
    7422: {"annotation": [["first started 5 yrs"]], "location": [["102 121"]]},
    8876: {"annotation": [["No shortness of breath"]], "location": [["481 483;533 552"]]},
    9027: {"annotation": [["recent URI"], ["nasal stuffines, rhinorrhea, for 3-4 days"]], "location": [["92 102"], ["123 164"]]},
    9938: {"annotation": [["irregularity with her cycles"], ["heavier bleeding"], ["changes her pad every couple hours"]],
           "location": [["89 117"], ["122 138"], ["368 402"]]},
    9973: {"annotation": [["gaining 10-15 lbs"]], "location": [["344 361"]]},
    10513: {"annotation": [["weight gain"], ["gain of 10-16lbs"]], "location": [["600 611"], ["607 623"]]},
    11551: {"annotation": [["seeing her son knows are not real"]], "location": [["386 400;443 461"]]},
    11677: {"annotation": [["saw him once in the kitchen after he died"]], "location": [["160 201"]]},
    12124: {"annotation": [["tried Ambien but it didnt work"]], "location": [["325 337;349 366"]]},
    12279: {"annotation": [["heard what she described as a party later than evening these things did not actually happen"]],
            "location": [["405 459;488 524"]]},
    12289: {"annotation": [["experienced seeing her son at the kitchen table these things did not actually happen"]],
            "location": [["353 400;488 524"]]},
    13238: {"annotation": [["SCRACHY THROAT"], ["RUNNY NOSE"]], "location": [["293 307"], ["321 331"]]},
    13297: {"annotation": [["without improvement when taking tylenol"], ["without improvement when taking ibuprofen"]],
            "location": [["182 221"], ["182 213;225 234"]]},
    13299: {"annotation": [["yesterday"], ["yesterday"]], "location": [["79 88"], ["409 418"]]},
    13845: {"annotation": [["headache global"], ["headache throughout her head"]], "location": [["86 94;230 236"], ["86 94;237 256"]]},
    14083: {"annotation": [["headache generalized in her head"]], "location": [["56 64;156 179"]]}
}

In [10]:
train = train.merge(features, on=['feature_num', 'case_num'], how='left')
train = train.merge(patient_notes, on=['pn_num', 'case_num'], how='left')
display(train.head())

,id,case_num,pn_num,feature_num,annotation,location,feature_text,pn_history
0,00016_000,0,16,0,[dad with recent heart attcak],[696 724],Family-history-of-MI-OR-Family-history-of-myoc...,HPI: 17yo M presents with palpitations. Patien...
1,00016_001,0,16,1,"[mom with ""thyroid disease]",[668 693],Family-history-of-thyroid-disorder,HPI: 17yo M presents with palpitations. Patien...
2,00016_002,0,16,2,[chest pressure],[203 217],Chest-pressure,HPI: 17yo M presents with palpitations. Patien...
3,00016_003,0,16,3,"[intermittent episodes, episode]","[70 91, 176 183]",Intermittent-symptoms,HPI: 17yo M presents with palpitations. Patien...
4,00016_004,0,16,4,[felt as if he were going to pass out],[222 258],Lightheaded,HPI: 17yo M presents with palpitations. Patien...


In [11]:
def lower_nested_list(nested_list):
    """將 list of list 中的字串轉為小寫"""
    return [[s.lower() for s in inner] for inner in nested_list]
    
for idx, value in corrections.items():
    train.loc[idx, 'annotation'] = value['annotation']
    train.loc[idx, 'location'] = value['location']
    
for col in train.columns:
    if col == 'annotation':
        train[col] = lower_nested_list(train[col])
    elif train[col].dtype == 'object' and col != 'location':
        train[col] = train[col].apply(lambda x: x.lower() if isinstance(x, str) else x)

display(train.head())

,id,case_num,pn_num,feature_num,annotation,location,feature_text,pn_history
0,00016_000,0,16,0,[dad with recent heart attcak],[696 724],family-history-of-mi-or-family-history-of-myoc...,hpi: 17yo m presents with palpitations. patien...
1,00016_001,0,16,1,"[mom with ""thyroid disease]",[668 693],family-history-of-thyroid-disorder,hpi: 17yo m presents with palpitations. patien...
2,00016_002,0,16,2,[chest pressure],[203 217],chest-pressure,hpi: 17yo m presents with palpitations. patien...
3,00016_003,0,16,3,"[intermittent episodes, episode]","[70 91, 176 183]",intermittent-symptoms,hpi: 17yo m presents with palpitations. patien...
4,00016_004,0,16,4,[felt as if he were going to pass out],[222 258],lightheaded,hpi: 17yo m presents with palpitations. patien...


In [12]:
train['annotation_length'] = train['annotation'].apply(len)

In [13]:
Fold = GroupKFold(n_splits=CFG.n_fold)
groups = train['pn_num'].values
for n, (train_index, val_index) in enumerate(Fold.split(train, train['location'], groups)):
    train.loc[val_index, 'fold'] = int(n)
train['fold'] = train['fold'].astype(int)
display(train.groupby('fold').size())

fold
0    2860
1    2860
2    2860
3    2860
4    2860
dtype: int64

In [14]:
OUTPUT_DIR = "./"

In [15]:
tokenizer = AutoTokenizer.from_pretrained(CFG.model, use_fast=True)
tokenizer.save_pretrained(OUTPUT_DIR+'tokenizer/')
CFG.tokenizer = tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:455: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
for text_col in ['pn_history']:
    pn_history_lengths = []
    tk0 = tqdm(patient_notes[text_col].fillna("").values, total=len(patient_notes))
    for text in tk0:
        length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
        pn_history_lengths.append(length)

for text_col in ['feature_text']:
    features_lengths = []
    tk0 = tqdm(features[text_col].fillna("").values, total=len(features))
    for text in tk0:
        length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
        features_lengths.append(length)
CFG.max_len = max(pn_history_lengths) + max(features_lengths) + 3 # cls & sep & sep

  0%|          | 0/42146 [00:00<?, ?it/s]

  0%|          | 0/143 [00:00<?, ?it/s]

In [ ]:
def prepare_input(cfg, text, feature_text):

    inputs = cfg.tokenizer(text, feature_text, 
                           add_special_tokens=True,
                           max_length=CFG.max_len,
                           padding="max_length",
                           return_offsets_mapping=False
                           )
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


def create_label(cfg, text, annotation_length, location_list):
    encoded = cfg.tokenizer(text,
                            add_special_tokens=True,
                            max_length=CFG.max_len,
                            padding="max_length",
                            return_offsets_mapping=True)
    offset_mapping = encoded['offset_mapping']
    ignore_idxes = np.where(np.array(encoded.sequence_ids()) != 0)[0]
    label = np.zeros(len(offset_mapping))
    label[ignore_idxes] = -1
    if annotation_length != 0:
        for location in location_list:
            for loc in [s.split() for s in location.split(';')]:
                start_idx = -1
                end_idx = -1
                start, end = int(loc[0]), int(loc[1])
                for idx in range(len(offset_mapping)):
                    if (start_idx == -1) & (start < offset_mapping[idx][0]):
                        start_idx = idx - 1
                    if (end_idx == -1) & (end <= offset_mapping[idx][1]):
                        end_idx = idx + 1
                if start_idx == -1:
                    start_idx = end_idx
                if (start_idx != -1) & (end_idx != -1):
                    label[start_idx:end_idx] = 1
    return torch.tensor(label, dtype=torch.float)


class TrainDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.feature_texts = df['feature_text'].values
        self.pn_historys = df['pn_history'].values
        self.annotation_lengths = df['annotation_length'].values
        self.locations = df['location'].values

    def __len__(self):
        return len(self.feature_texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, 
                               self.pn_historys[item], 
                               self.feature_texts[item])
        label = create_label(self.cfg, 
                             self.pn_historys[item], 
                             self.annotation_lengths[item], 
                             self.locations[item])
        return inputs, label

In [18]:
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel(self.config)

        self.layer_weights = nn.Parameter(torch.ones(self.config.num_hidden_layers + 1) / (self.config.num_hidden_layers + 1))

        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, 1)
        self._init_weights(self.fc)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def feature(self, inputs):
        outputs = self.model(**inputs)
        hidden_states = outputs.hidden_states 
        weighted_sum = 0
        for i, layer_hidden in enumerate(hidden_states):
            weighted_sum += self.layer_weights[i] * layer_hidden
        return weighted_sum

    def forward(self, inputs):
        x = self.feature(inputs)
        x = self.fc_dropout(x)
        output = self.fc(x)  # (batch_size, seq_len, 1)
        return output.squeeze(-1)


In [19]:
class AverageMeter:
    """Tracks and averages metrics (e.g., loss)."""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.sum = 0
        self.count = 0
        self.avg = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device):
    model.train()
    losses = AverageMeter()
    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
    global_step = 0

    for step, (inputs, labels) in enumerate(train_loader):
        inputs = {k: v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)

        with torch.cuda.amp.autocast(enabled=CFG.apex):
            y_preds = model(inputs)
            loss = criterion(y_preds.view(-1, 1), labels.view(-1, 1))
            loss = torch.masked_select(loss, labels.view(-1, 1) != -1).mean()

        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps

        losses.update(loss.item(), labels.size(0))
        scaler.scale(loss).backward()

        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)

        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
            scheduler.step()

        if step % CFG.print_freq == 0 or step == len(train_loader) - 1:
            current_lr = scheduler.get_last_lr()[0]
            print(f"Epoch [{epoch + 1}] Step [{step}/{len(train_loader)}] "
                  f"Loss: {losses.val:.4f} (Avg: {losses.avg:.4f}) "
                  f"GradNorm: {grad_norm:.4f} LR: {current_lr:.8f}")

    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    model.eval()
    losses = AverageMeter()
    preds = []

    for step, (inputs, labels) in enumerate(valid_loader):
        inputs = {k: v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)

        with torch.no_grad():
            y_preds = model(inputs)
            loss = criterion(y_preds.view(-1, 1), labels.view(-1, 1))
            loss = torch.masked_select(loss, labels.view(-1, 1) != -1).mean()

        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps

        losses.update(loss.item(), labels.size(0))
        preds.append(torch.sigmoid(y_preds).cpu().numpy())

        if step % CFG.print_freq == 0 or step == len(valid_loader) - 1:
            print(f"[EVAL] Step [{step}/{len(valid_loader)}] "
                  f"Loss: {losses.val:.4f} (Avg: {losses.avg:.4f})")

    predictions = np.concatenate(preds, axis=0)
    return losses.avg, predictions


def inference_fn(test_loader, model, device):
    model.eval()
    model.to(device)
    preds = []

    for inputs in tqdm(test_loader, total=len(test_loader)):
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            y_preds = model(inputs)

        preds.append(torch.sigmoid(y_preds).cpu().numpy())

    predictions = np.concatenate(preds, axis=0)
    return predictions

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def train_loop(folds, fold):
    # ====================================================
    # loader
    # ====================================================
    train_folds = folds[folds['fold'] != fold].reset_index(drop=True)
    valid_folds = folds[folds['fold'] == fold].reset_index(drop=True)
    valid_texts = valid_folds['pn_history'].values
    valid_labels = create_labels_for_scoring(valid_folds)
    
    train_dataset = TrainDataset(CFG, train_folds)
    valid_dataset = TrainDataset(CFG, valid_folds)

    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(CFG, config_path=None, pretrained=True)
    torch.save(model.config, OUTPUT_DIR+'config.pth')
    model.to(device)
    
    def get_optimizer_params(model, lr, weight_decay=0.0):
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay)],
             'lr': lr, 'weight_decay': weight_decay},
            {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay)],
             'lr': lr, 'weight_decay': 0.0},
            {'params': [p for n, p in model.named_parameters() if "model" not in n],
             'lr': lr, 'weight_decay': 0.0}
        ]
        return optimizer_parameters

    optimizer_parameters = get_optimizer_params(model,
                                                lr = CFG.lr,
                                                weight_decay=CFG.weight_decay)
    optimizer = AdamW(optimizer_parameters, lr=CFG.lr, eps=CFG.eps)
    
    def get_scheduler(cfg, optimizer, num_train_steps):
        scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=0.5
        )
        return scheduler
    
    num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)

    # ====================================================
    # loop
    # ====================================================
    criterion = nn.BCEWithLogitsLoss(reduction="none")
    
    best_score = 0.

    for epoch in range(CFG.epochs):

        # train
        print("Starting training")
        avg_loss = train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device)

        # eval
        print("Starting evaluation")
        avg_val_loss, predictions = valid_fn(valid_loader, model, criterion, device)
        predictions = predictions.reshape((len(valid_folds), CFG.max_len))
        
        # scoring
        char_probs = get_char_probs(valid_texts, predictions, CFG.tokenizer)
        results = get_results(char_probs, th=0.5)
        preds = get_predictions(results)
        score = get_score(valid_labels, preds)
        
        if best_score <= score:
            best_score = score
            torch.save({'model': model.state_dict(),
                        'predictions': predictions},
                        OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth")

    predictions = torch.load(OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth", 
                             map_location=torch.device('cpu'))['predictions']
    valid_folds[[i for i in range(CFG.max_len)]] = predictions

    torch.cuda.empty_cache()
    gc.collect()
    
    return valid_folds

In [22]:
def get_result(oof_df):
    labels = create_labels_for_scoring(oof_df)
    predictions = oof_df[[i for i in range(CFG.max_len)]].values
    char_probs = get_char_probs(oof_df['pn_history'].values, predictions, CFG.tokenizer)
    results = get_results(char_probs, th=0.5)
    preds = get_predictions(results)
    score = get_score(labels, preds)

    print(f"F1 Score: {score:.5f}")
    return score


oof_df = pd.DataFrame()
for fold in range(CFG.n_fold):
    if fold in CFG.trn_fold:
        print(f"\n### Start training Fold {fold} ###")
        _oof_df = train_loop(train, fold)
        score = get_result(_oof_df)
        print(f"[Fold {fold}] Done - F1 Score: {score:.5f}")
        oof_df = pd.concat([oof_df, _oof_df], axis=0)
        
oof_df = oof_df.reset_index(drop=True)
final_score = get_result(oof_df)

oof_path = OUTPUT_DIR + "oof_df.pkl"
oof_df.to_pickle(oof_path)



### Start training Fold 0 ###
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | fals

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2Model: ['mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Starting training
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingfac

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:40: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.


Epoch [1] Step [0/953] Loss: 0.7271 (Avg: 0.7271) GradNorm: inf LR: 0.00002000
Epoch [1] Step [100/953] Loss: 0.0500 (Avg: 0.1409) GradNorm: 967.7938 LR: 0.00001998
Epoch [1] Step [200/953] Loss: 0.0627 (Avg: 0.0963) GradNorm: 8641.9834 LR: 0.00001991
Epoch [1] Step [300/953] Loss: 0.0144 (Avg: 0.0764) GradNorm: 1256.1755 LR: 0.00001980
Epoch [1] Step [400/953] Loss: 0.0091 (Avg: 0.0646) GradNorm: 2330.3284 LR: 0.00001965
Epoch [1] Step [500/953] Loss: 0.0245 (Avg: 0.0573) GradNorm: 2698.8206 LR: 0.00001946
Epoch [1] Step [600/953] Loss: 0.0138 (Avg: 0.0518) GradNorm: 2190.2319 LR: 0.00001923
Epoch [1] Step [700/953] Loss: 0.0209 (Avg: 0.0476) GradNorm: 1951.9261 LR: 0.00001895
Epoch [1] Step [800/953] Loss: 0.0088 (Avg: 0.0444) GradNorm: 1344.4316 LR: 0.00001864
Epoch [1] Step [900/953] Loss: 0.0125 (Avg: 0.0418) GradNorm: 1591.4851 LR: 0.00001829
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlo

...

Starting training
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingfac

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2Model: ['mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Starting training
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingfac

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2Model: ['mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Starting training
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingfac